# *Intro to Postgress*

### Connecting to Postgress

In [1]:
# Before running below commandstype following into terminal
# psql dq
# When prompted with password enter password

import psycopg2
conn = psycopg2.connect("dbname=dq user=dq host=localhost password='dq'")
print(conn)
conn.close()

<connection object at 0x7f7fc06273d8; dsn: 'user=dq password=xxx dbname=dq host=localhost', closed: 0>


### Interacting the Database

In [2]:
conn = psycopg2.connect("dbname=dq user=dq host=localhost password='dq'")
query = "SELECT * FROM notes"
cur = conn.cursor()
cur.execute(query)

notes = cur.fetchall()
print(notes)
conn.close()

[]


### Creating a Table

In [3]:
conn = psycopg2.connect("dbname=dq user=dq host=localhost password='dq'")

cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS users") # Dropping table if already exists

# Creating table users (id, email, name and address)
cur.execute("CREATE TABLE users(id INTEGER PRIMARY KEY, email TEXT, name TEXT, address TEXT);") 

# Displaying all the Databases
cur.execute("SELECT datname FROM pg_database WHERE datistemplate = false;")
print(cur.fetchall())

# Displaying all the tables created by us
cur.execute("SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' and schemaname != 'information_schema';")
cur.fetchall()

[('postgres',), ('naveen',), ('dq',)]


[('public', 'notes', 'naveen', None, False, False, False, False),
 ('public', 'users', 'dq', None, True, False, False, False)]

### SQL Transactions

In [4]:
conn = psycopg2.connect("dbname=dq user=dq host=localhost password='dq'")

cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS users") # Dropping table if already exists

# Creating table users (id, email, name and address)
cur.execute("CREATE TABLE users(id INTEGER PRIMARY KEY, email TEXT, name TEXT, address TEXT);") 

conn.commit()

# Displaying all the tables created by us
cur.execute("SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' and schemaname != 'information_schema';")
print(cur.fetchall())

conn.close()

[('public', 'notes', 'naveen', None, False, False, False, False), ('public', 'users', 'dq', None, True, False, False, False)]


### Inserting the Data

In [5]:
conn = psycopg2.connect("dbname=dq user=dq host=localhost password='dq'")

cur = conn.cursor()

# Getting Information Schema of a Table
cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'users';")
cur.fetchall()

[('id',), ('email',), ('name',), ('address',)]

In [6]:
import csv

import psycopg2
conn = psycopg2.connect("user=dq dbname=dq host=localhost password='dq'")
cur = conn.cursor()
cur.execute("DELETE FROM users;")

with open('user_accounts.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    rows = [x for x in reader]

for row in rows:
    cur.execute("INSERT INTO users VALUES(%s, %s, %s, %s)", row)
    
conn.commit()
    
cur.execute("SELECT * FROM users")
users = cur.fetchall()
conn.close()

users[:5]

[(0,
  'cbenjamin@yahoo.com',
  'Joseph Kirby',
  '3594 Fox Ford Apt. 192 West Kristen GA 22838-8977'),
 (1,
  'morganlopez@matthews-hickman.com',
  'Erin Figueroa',
  '64763 Li Meadows Apt. 554 New Marcoton MA 99016-6159'),
 (2,
  'ypark@russo.biz',
  'Leon Matthews',
  '91144 Hamilton Manors Suite 421 Ronaldland WA 98705'),
 (3,
  'lferguson@hotmail.com',
  'Nicole Spencer',
  '798 Hawkins Mountains Lindseybury MH 53952-7424'),
 (4,
  'brownanna@gmail.com',
  'Jacob Webb',
  '5388 Whitney Stream Rowlandville AL 35533')]

### Copying the Data

In [7]:
conn = psycopg2.connect("user=dq dbname=dq host=localhost password='dq'")
cur = conn.cursor()
cur.execute("DELETE FROM users;")

with open('user_accounts.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'users', sep=",")
conn.commit()
    
cur.execute("SELECT * FROM users")
users = cur.fetchall()
conn.close()

users[:5]

[(0,
  'cbenjamin@yahoo.com',
  'Joseph Kirby',
  '3594 Fox Ford Apt. 192 West Kristen GA 22838-8977'),
 (1,
  'morganlopez@matthews-hickman.com',
  'Erin Figueroa',
  '64763 Li Meadows Apt. 554 New Marcoton MA 99016-6159'),
 (2,
  'ypark@russo.biz',
  'Leon Matthews',
  '91144 Hamilton Manors Suite 421 Ronaldland WA 98705'),
 (3,
  'lferguson@hotmail.com',
  'Nicole Spencer',
  '798 Hawkins Mountains Lindseybury MH 53952-7424'),
 (4,
  'brownanna@gmail.com',
  'Jacob Webb',
  '5388 Whitney Stream Rowlandville AL 35533')]

# *Creating Tables*

Choosing the correct Data Type will provide benefit of a reduced table size, and guard against corrupted data.

### Describing a Table

In [10]:
conn = psycopg2.connect("user=dq dbname=dq host=localhost password='dq'")
cur = conn.cursor()
cur.execute("SELECT * FROM users LIMIT 0;")
print(cur.description)
conn.close()

(Column(name='id', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='email', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None), Column(name='name', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None), Column(name='address', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None))


Now we will work with `ign.csv` file.

In [14]:
import pandas as pd
df = pd.read_csv('ign.csv')
df.head()

,id,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day
0,5249979066121302517,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,Y,2012,9,12
1,7399589116837456607,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,Y,2012,9,12
2,582057716445789124,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,N,2012,9,12
3,1087608058291172412,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,N,2012,9,11
4,7018639715332314491,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,N,2012,9,11


In [28]:
df['score'].unique()

array([ 9. ,  8.5,  7. ,  3. ,  7.5,  6.5,  8. ,  5.5,  8.7,  4.9,  9.6,
        7.3,  9.2,  7.9,  6.9,  9.1,  9.5,  6. ,  2.5,  8.2,  8.8,  4. ,
        3.9,  7.1,  5.8,  7.2,  8.3,  7.6,  9.3,  6.8,  2.3,  4.5,  5. ,
        7.4,  6.6,  6.7,  6.2,  5.3,  9.8,  9.4,  7.8,  8.4,  5.2,  4.7,
        6.3,  7.7,  6.4,  4.1,  8.6,  8.1,  5.6,  8.9,  4.2,  3.8,  5.9,
        2. ,  1. ,  9.7,  5.4,  4.3,  3.7,  3.5,  2.9,  3.2,  5.7,  0.7,
        6.1,  1.8,  4.6,  2.8,  4.8, 10. ,  5.1,  4.4,  3.6,  1.5,  3.4,
        2.7,  3.3,  2.2,  2.1,  2.6,  1.9,  3.1,  1.3,  9.9,  2.4,  1.2,
        1.1,  0.5,  1.7,  1.4,  0.8])

### Adding the id Field

In [12]:
conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

# We will use BIGINT data type for id since it takes appx. 63 bits/8 bytes of storage
cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("CREATE TABLE ign_reviews (id BIGINT PRIMARY KEY);")
cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
conn.commit()
print(cur.fetchall())
conn.close()

[('id',)]


### Finding the Max Length

In [1]:
import csv 

with open('ign.csv', 'r') as f:
    next(f)
    reader = csv.reader(f)
    print(list(reader)[:5])

[['5249979066121302517', 'Amazing', 'LittleBigPlanet PS Vita', '/games/littlebigplanet-vita/vita-98907', 'PlayStation Vita', '9.0', 'Platformer', 'Y', '2012', '9', '12'], ['7399589116837456607', 'Amazing', 'LittleBigPlanet PS Vita -- Marvel Super Hero Edition', '/games/littlebigplanet-ps-vita-marvel-super-hero-edition/vita-20027059', 'PlayStation Vita', '9.0', 'Platformer', 'Y', '2012', '9', '12'], ['582057716445789124', 'Great', 'Splice: Tree of Life', '/games/splice/ipad-141070', 'iPad', '8.5', 'Puzzle', 'N', '2012', '9', '12'], ['1087608058291172412', 'Great', 'NHL 13', '/games/nhl-13/xbox-360-128182', 'Xbox 360', '8.5', 'Sports', 'N', '2012', '9', '11'], ['7018639715332314491', 'Great', 'NHL 13', '/games/nhl-13/ps3-128181', 'PlayStation 3', '8.5', 'Sports', 'N', '2012', '9', '11']]


In [13]:
import csv 

with open('ign.csv', 'r') as f:
    next(f)
    reader = csv.reader(f)
    unique_words_in_score = set([row[1] for row in reader])
    
print(unique_words_in_score)

# Getting length of the longest word
max_score = max([len(x) for x in unique_words_in_score])
max_score

{'Disaster', 'Good', 'Unbearable', 'Great', 'Amazing', 'Masterpiece', 'Bad', 'Mediocre', 'Painful', 'Okay', 'Awful'}
11


### Max String-like Datatypes

In [21]:
conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

# We will use varchar(n) data type for score_phrase since it takes MAX. 11 characters and 
# it will provide for data validation.

cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("CREATE TABLE ign_reviews (id BIGINT PRIMARY KEY, score_phrase varchar(11));")
conn.commit()

cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',)]


### Creating the Other String Fields

In [22]:
with open('ign.csv', 'r') as f:
    next(f)
    reader = csv.reader(f)    
    rows = [row for row in reader]
    
len_characters_for_platform = [len(row[4]) for row in rows]
max_platform = max(len_characters_for_platform)

len_characters_for_genre = [len(row[6]) for row in rows]
max_genre = max(len_characters_for_genre)

print(max_platform)
print(max_genre)

20
25


In [23]:
conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

# We will use varchar(n) data type for platform and genre since it takes MAX. 20 and 25 characters resp. and 
# it will provide for data validation.
cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
genre VARCHAR(25));""")
conn.commit()

cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('genre',)]


### Float-like types

In [24]:
conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

# We will use DECIMAL(precision, scale) data type for score. Since, it takes 3 digits total & 1 digit after decimal
cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
genre VARCHAR(25),
score DECIMAL(3, 1));""")
conn.commit()

cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('genre',), ('score',)]


### Boolean Types

In [26]:
conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

# We will use BOOLEAN data type for editors_choice. Since, it takes boolean value of Y or N only
cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
score DECIMAL(3, 1),
genre VARCHAR(25),
editors_choice BOOLEAN);""")
conn.commit()

cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('score',), ('genre',), ('editors_choice',)]


### Date Type

In [29]:
conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

# We will use DATE data type for release_date. And, combine the 3 columns into one using INSERT 
# with string interpolation
cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
score DECIMAL(3, 1),
genre VARCHAR(25),
editors_choice BOOLEAN,
release_date DATE);""")

from datetime import date
import csv

with open('ign.csv', 'r') as f:
    next(f)
    reader = csv.reader(f)
    for row in reader:
        updated_row = row[:8]
        updated_row.append(date(int(row[8]), int(row[9]), int(row[10])))
        cur.execute("INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", updated_row)

conn.commit()

cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('score',), ('genre',), ('editors_choice',), ('release_date',)]


# Loading and Extracting Data with Tables

In [25]:
# Getting that old table (old_ign_reviews) with seperate datetime elements.

conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS old_ign_reviews")
cur.execute("""
CREATE TABLE old_ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
score DECIMAL(3, 1),
genre VARCHAR(25),
editors_choice BOOLEAN,
release_year smallint,
release_month smallint,
release_day smallint);""")

from datetime import date
import csv

with open('ign.csv', 'r') as f:
    cur.copy_expert('COPY old_ign_reviews FROM STDIN WITH CSV HEADER', f)

conn.commit()

cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'old_ign_reviews';")
print(cur.fetchall())

cur.execute("SELECT * FROM old_ign_reviews LIMIT 2;")
print(cur.fetchall())
conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('score',), ('genre',), ('editors_choice',), ('release_year',), ('release_month',), ('release_day',)]
[(5249979066121302517, 'Amazing', 'LittleBigPlanet PS Vita', '/games/littlebigplanet-vita/vita-98907', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, 2012, 9, 12), (7399589116837456607, 'Amazing', 'LittleBigPlanet PS Vita -- Marvel Super Hero Edition', '/games/littlebigplanet-ps-vita-marvel-super-hero-edition/vita-20027059', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, 2012, 9, 12)]


In [3]:
# ign1.csv contains new csv file with single column for Date

import pandas as pd
df1 = pd.read_csv('ign1.csv')
print(df1.shape)
df1.head()

(18625, 9)


,id,score_phrase,title,url,platform,score,genre,editors_choice,release_date
0,5249979066121302517,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,Y,2012-09-12
1,7399589116837456607,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,Y,2012-09-12
2,582057716445789124,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,N,2012-09-12
3,1087608058291172412,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,N,2012-09-11
4,7018639715332314491,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,N,2012-09-11


### Prepared Insert with multiple execute and insert

In [6]:
import psycopg2
import csv

conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
score DECIMAL(3, 1),
genre VARCHAR(25),
editors_choice BOOLEAN,
release_date DATE);""")

from datetime import date
import csv

with open('ign1.csv', 'r') as f:
    next(f)
    reader = csv.reader(f)
    for row in reader:
        cur.execute("INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", row)

conn.commit()

cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())

cur.execute("""
SELECT id, score_phrase, platform, score, genre, editors_choice, release_date 
    from ign_reviews 
    limit 5;""")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('score',), ('genre',), ('editors_choice',), ('release_date',)]
[(5249979066121302517, 'Amazing', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12)), (7399589116837456607, 'Amazing', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12)), (582057716445789124, 'Great', 'iPad', Decimal('8.5'), 'Puzzle', False, datetime.date(2012, 9, 12)), (1087608058291172412, 'Great', 'Xbox 360', Decimal('8.5'), 'Sports', False, datetime.date(2012, 9, 11)), (7018639715332314491, 'Great', 'PlayStation 3', Decimal('8.5'), 'Sports', False, datetime.date(2012, 9, 11))]


### Faster Inserts with mogrify

In [7]:
conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
score DECIMAL(3, 1),
genre VARCHAR(25),
editors_choice BOOLEAN,
release_date DATE);""")

import csv

with open('ign1.csv', 'r') as f:
    next(f)
    reader = csv.reader(f)
    for row in reader:
        mogrified = [cur.mogrify("(%s, %s, %s, %s, %s, %s, %s, %s, %s)", row).decode('utf-8') for row in reader]

mogrified_values = ",".join(mogrified)
cur.execute("INSERT INTO ign_reviews VALUES " + mogrified_values)
conn.commit()

cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())

cur.execute("""
SELECT id, score_phrase, platform, score, genre, editors_choice, release_date 
    from ign_reviews 
    limit 5;""")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('score',), ('genre',), ('editors_choice',), ('release_date',)]
[(7399589116837456607, 'Amazing', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12)), (582057716445789124, 'Great', 'iPad', Decimal('8.5'), 'Puzzle', False, datetime.date(2012, 9, 12)), (1087608058291172412, 'Great', 'Xbox 360', Decimal('8.5'), 'Sports', False, datetime.date(2012, 9, 11)), (7018639715332314491, 'Great', 'PlayStation 3', Decimal('8.5'), 'Sports', False, datetime.date(2012, 9, 11)), (4355693531291048099, 'Good', 'Macintosh', Decimal('7.0'), 'Strategy', False, datetime.date(2012, 9, 11))]


### Advanced COPY Statement with copy_expert()

In [10]:
# The `cur` object is provided for you.
conn = psycopg2.connect("dbname=dq user=dq host=localhost password=dq")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
score DECIMAL(3, 1),
genre VARCHAR(25),
editors_choice BOOLEAN,
release_date DATE);""")

with open('ign1.csv', 'r') as f:
    cur.copy_expert("COPY ign_reviews FROM STDIN WITH CSV HEADER", f)
    
conn.commit()

cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())

cur.execute("""
SELECT id, score_phrase, platform, score, genre, editors_choice, release_date 
    from ign_reviews 
    limit 5;""")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('score',), ('genre',), ('editors_choice',), ('release_date',)]
[(5249979066121302517, 'Amazing', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12)), (7399589116837456607, 'Amazing', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12)), (582057716445789124, 'Great', 'iPad', Decimal('8.5'), 'Puzzle', False, datetime.date(2012, 9, 12)), (1087608058291172412, 'Great', 'Xbox 360', Decimal('8.5'), 'Sports', False, datetime.date(2012, 9, 11)), (7018639715332314491, 'Great', 'PlayStation 3', Decimal('8.5'), 'Sports', False, datetime.date(2012, 9, 11))]


### Which method is faster? Finding using time.time()

In [21]:
import time

conn = psycopg2.connect("dbname=dq user=dq host=localhost password=dq")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
score DECIMAL(3, 1),
genre VARCHAR(25),
editors_choice BOOLEAN,
release_date DATE);""")

# Multiple single insert statements.
start = time.time()
with open('ign1.csv', 'r') as f:
    next(f)
    reader = csv.reader(f)
    for row in reader:
        cur.execute(
            "INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
            row
        )
conn.rollback()
print("Single statment insert: ", time.time() - start)
        
# Multiple mogrify insert.
# start = time.time()
# with open('ign1.csv', 'r') as f:
#    next(f)
#    reader = csv.reader(f)
#    mogrified = [ 
#        cur.mogrify("(%s, %s, %s, %s, %s, %s, %s, %s, %s)", row).decode('utf-8')
#        for row in reader
#    ] 
#    mogrified_values = ",".join(mogrified) 
#    cur.execute('INSERT INTO ign_reviews VALUES ' + mogrified_values)
# conn.rollback()
# print("Multiple mogrify insert: ", time.time() - start)

        
# Copy expert method.
# start = time.time()
# with open('ign1.csv', 'r') as f:
#    cur.copy_expert('COPY ign_reviews FROM STDIN WITH CSV HEADER', f)
# conn.rollback()
# print("Copy expert method: ", time.time() - start)

conn.close()

Single statment insert:  2.2308835983276367


### Extracting Table to CSV File using copy_expert()

In [22]:
import pandas as pd
conn = psycopg2.connect("dbname=dq user=dq host=localhost password=dq")
cur = conn.cursor()

with open('ign2.csv', 'w') as f:
    cur.copy_expert("COPY ign_reviews TO STDOUT WITH CSV HEADER", f)
    
conn.commit()
conn.close()

# Now ign1.csv and ign2.csv both are with one column for date
df = pd.read_csv('ign2.csv')
print(df.shape)
df.head(5)

(18625, 9)


,id,score_phrase,title,url,platform,score,genre,editors_choice,release_date
0,5249979066121302517,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,t,2012-09-12
1,7399589116837456607,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,t,2012-09-12
2,582057716445789124,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,f,2012-09-12
3,1087608058291172412,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,f,2012-09-11
4,7018639715332314491,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,f,2012-09-11


In [28]:
conn = psycopg2.connect("dbname=dq user=dq host=localhost password=dq")
cur = conn.cursor()

with open('old_ign_reviews.csv', 'w') as f:
    cur.copy_expert("COPY old_ign_reviews TO STDOUT WITH CSV HEADER", f)
    
conn.commit()
conn.close()

# Now old_ign_reviews.csv has seperate datetime elements 
df = pd.read_csv('old_ign_reviews.csv')
print(df.shape)
df.head(5)

(18625, 11)


,id,score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day
0,5249979066121302517,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,9.0,Platformer,t,2012,9,12
1,7399589116837456607,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,9.0,Platformer,t,2012,9,12
2,582057716445789124,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,8.5,Puzzle,f,2012,9,12
3,1087608058291172412,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,8.5,Sports,f,2012,9,11
4,7018639715332314491,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,8.5,Sports,f,2012,9,11


### Transform an old table to a new table using copy_expert

In [30]:
import csv
conn = psycopg2.connect("user=dq dbname=dq host=localhost password=dq")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
score DECIMAL(3, 1),
genre VARCHAR(25),
editors_choice BOOLEAN,
release_date DATE);""")

with open('old_ign_reviews.csv', 'r+') as f:
    cur.copy_expert("COPY old_ign_reviews TO STDOUT WITH CSV HEADER", f)
    
    f.seek(0)
    reader = csv.reader(f)
    next(reader)
    
    for row in reader:
        updated_row = row[:8]
        updated_row.append(date(int(row[8]), int(row[9]), int(row[10])))
        cur.execute("INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", updated_row)
    conn.commit()
    
# Printing Column Names
cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())
print()

# Printing First two rows
cur.execute("SELECT * FROM ign_reviews LIMIT 2;")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('score',), ('genre',), ('editors_choice',), ('release_date',)]

[(5249979066121302517, 'Amazing', 'LittleBigPlanet PS Vita', '/games/littlebigplanet-vita/vita-98907', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12)), (7399589116837456607, 'Amazing', 'LittleBigPlanet PS Vita -- Marvel Super Hero Edition', '/games/littlebigplanet-ps-vita-marvel-super-hero-edition/vita-20027059', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12))]


### Insert  Into Table with SELECT

In [32]:
# The `cur` object is provided for you.
conn = psycopg2.connect("dbname=dq user=dq host=localhost password=dq")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS ign_reviews")
cur.execute("""
CREATE TABLE ign_reviews (
id BIGINT PRIMARY KEY, 
score_phrase varchar(11),
title TEXT,
url TEXT,
platform VARCHAR(20),
score DECIMAL(3, 1),
genre VARCHAR(25),
editors_choice BOOLEAN,
release_date DATE);""")

cur.execute("""
INSERT INTO ign_reviews (
    id, score_phrase, title, url, platform,
    score, genre, editors_choice, release_date) 
SELECT 
id, score_phrase, title, url, 
platform, score, genre, editors_choice, 
to_date(release_day || '-' || release_month ||
'-' || release_year, 'DD-MM-YYYY') as release_date FROM old_ign_reviews;
""")

conn.commit()

# Printing Column Names
cur.execute("SELECT COLUMN_NAME FROM information_schema.COLUMNS WHERE TABLE_NAME = 'ign_reviews';")
print(cur.fetchall())
print()

# Printing First two rows
cur.execute("SELECT * FROM ign_reviews LIMIT 2;")
print(cur.fetchall())

conn.close()

[('id',), ('score_phrase',), ('title',), ('url',), ('platform',), ('score',), ('genre',), ('editors_choice',), ('release_date',)]

[(5249979066121302517, 'Amazing', 'LittleBigPlanet PS Vita', '/games/littlebigplanet-vita/vita-98907', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12)), (7399589116837456607, 'Amazing', 'LittleBigPlanet PS Vita -- Marvel Super Hero Edition', '/games/littlebigplanet-ps-vita-marvel-super-hero-edition/vita-20027059', 'PlayStation Vita', Decimal('9.0'), 'Platformer', True, datetime.date(2012, 9, 12))]
